<a href="https://colab.research.google.com/github/cbeckler/final_project/blob/kp_etl/Balanced_random_forest_NaN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:14 http://security.ubuntu.com/ubuntu

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-13 04:53:20--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.48MB/s    in 0.2s    

2023-01-13 04:53:21 (5.48 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
# start spark session 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("random_forest_NAN").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#connect to database

import pandas as pd

conn_path = "/content/drive/MyDrive/data/db_conn_info.csv"

conn_info = pd.read_csv(conn_path)

db_name = conn_info['db_name'][0]

db_pw = conn_info['password'][0]

db_host = conn_info['host'][0]

db_port = conn_info['port'][0]

db_user = conn_info['user'][0]

In [7]:
from sqlalchemy import create_engine

engine = create_engine(f'postgresql://{db_name}:{db_pw}@{db_host}:{db_port}/{db_user}')

In [9]:
# # import model dataset

# df = pd.read_sql("select * from null_model_dataset", engine)

# df.head()

In [20]:
df=pd.read_csv('/content/drive/MyDrive/data/null_model_dataset.csv')

In [21]:
df

,Index,FIPS,recent_trend,PM25_max_pred,PM25_med_pred,PM25_mean_pred,O3_max_pred,O3_med_pred,O3_mean_pred,PM25_max_pred_2001,...,O3_med_pred_2005,O3_med_pred_2006,O3_med_pred_2007,O3_med_pred_2008,O3_med_pred_2009,O3_med_pred_2010,O3_med_pred_2011,O3_med_pred_2012,O3_med_pred_2013,O3_med_pred_2014
0,0,0,falling,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1001,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1003,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1005,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1007,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,2667,56037,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2668,2668,56039,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2669,2669,56041,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2670,2670,56043,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2672 entries, 0 to 2671
Data columns (total 93 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Index                2672 non-null   int64  
 1   FIPS                 2672 non-null   int64  
 2   recent_trend         2672 non-null   object 
 3   PM25_max_pred        1048 non-null   float64
 4   PM25_med_pred        1048 non-null   float64
 5   PM25_mean_pred       1048 non-null   float64
 6   O3_max_pred          1048 non-null   float64
 7   O3_med_pred          1048 non-null   float64
 8   O3_mean_pred         1048 non-null   float64
 9   PM25_max_pred_2001   1048 non-null   float64
 10  PM25_max_pred_2002   1048 non-null   float64
 11  PM25_max_pred_2003   1048 non-null   float64
 12  PM25_max_pred_2004   1048 non-null   float64
 13  PM25_max_pred_2005   1048 non-null   float64
 14  PM25_max_pred_2006   1048 non-null   float64
 15  PM25_max_pred_2007   1048 non-null   f

In [23]:
col_list = [name for name in df.columns]

col_list = col_list[3:]

for col in col_list:

  df[col].fillna(df[col].mean(), inplace=True)

In [24]:
df

,Index,FIPS,recent_trend,PM25_max_pred,PM25_med_pred,PM25_mean_pred,O3_max_pred,O3_med_pred,O3_mean_pred,PM25_max_pred_2001,...,O3_med_pred_2005,O3_med_pred_2006,O3_med_pred_2007,O3_med_pred_2008,O3_med_pred_2009,O3_med_pred_2010,O3_med_pred_2011,O3_med_pred_2012,O3_med_pred_2013,O3_med_pred_2014
0,0,0,falling,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
1,1,1001,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
2,2,1003,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
3,3,1005,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
4,4,1007,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,2667,56037,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
2668,2668,56039,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
2669,2669,56041,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
2670,2670,56043,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843


In [25]:
import numpy as np
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
from imblearn.over_sampling import SMOTE

In [26]:
X = df.drop(['Index', 'FIPS', 'recent_trend'], axis=1)
y = df['recent_trend']

In [27]:
y.value_counts()

stable     2429
falling     200
rising       43
Name: recent_trend, dtype: int64

In [28]:
#Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1,
                                                    stratify=y)
scaler =StandardScaler().fit(X_train)

X_train_scaled =scaler.transform(X_train)
X_test_scaled =scaler.transform(X_test)

In [29]:
# Resampling training data with SMOTE
X_resampled, y_resampled = SMOTE(
    random_state=1,sampling_strategy='auto').fit_resample(
    X_train_scaled, y_train)

print(f'y_resampled: {Counter(y_resampled)}')

y_resampled: Counter({'stable': 1822, 'falling': 1822, 'rising': 1822})


In [31]:
from imblearn.ensemble import BalancedRandomForestClassifier
brc=BalancedRandomForestClassifier(random_state=1, n_estimators=100).fit(X_resampled, y_resampled )

In [38]:
y_pred_train= brc.predict(X_train_scaled)

In [39]:
balanced_accuracy_score(y_train, y_pred_train)

0.6085117087449688

In [32]:
y_pred= brc.predict(X_test_scaled)

In [33]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)


0.3612850082372323

In [40]:

cm=confusion_matrix(y_test, y_pred)
cm

array([[ 35,   0,  15],
       [  7,   0,   4],
       [368,   6, 233]])

In [41]:
brc_df=pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
brc_df

,Prediction,Actual
0,stable,stable
1,stable,stable
2,stable,stable
3,stable,stable
4,falling,stable
...,...,...
663,falling,stable
664,falling,stable
665,stable,stable
666,falling,stable


In [42]:
print(brc_df.Prediction.value_counts())
print(brc_df.Actual.value_counts())

falling    410
stable     252
rising       6
Name: Prediction, dtype: int64
stable     607
falling     50
rising      11
Name: Actual, dtype: int64


In [43]:
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

    falling       0.09      0.70      0.39      0.15      0.52      0.28        50
     rising       0.00      0.00      0.99      0.00      0.00      0.00        11
     stable       0.92      0.38      0.69      0.54      0.51      0.26       607

avg / total       0.85      0.40      0.67      0.50      0.51      0.25       668

